In [ ]:
!pip install crewai crewai_tools

In [39]:
from arcadepy import Arcade
from dotenv import load_dotenv
import logging
from langchain_arcade._utilities import tool_definition_to_pydantic_model
from crewai_arcade.structed import StructuredTool

load_dotenv()

arcade_client = Arcade()

def tool_util(arcade_client, tool_id="Google_CreateEvent"):
    """
    Creates a StructuredTool for Google Calendar event creation.
    
    Args:
        arcade_client: Arcade client instance
        tool_id: ID of the Google Calendar event tool
        
    Returns:
        StructuredTool: Configured tool for creating Google Calendar events
    """
    # Wrapper function to handle Google Calendar event creation
    def tool_util(*args, **kwargs):
        print(args)
        print(kwargs)
        logging.info(f"Event created: {kwargs}")
        logging.info(f"Event created: {kwargs}")
        return 'Event created'
    
    # Get tool definition from Arcade
    tool_def = arcade_client.tools.get(tool_id=tool_id)
    
    # Extract tool properties
    description = tool_def.description
    tool_name = tool_id
    args_schema = tool_definition_to_pydantic_model(tool_def)
    
    # Create and return the structured tool
    return StructuredTool.from_function(
        func=tool_util,
        name=tool_name,
        description=description,
        args_schema=args_schema
    )

# Create the tool instance
google_tool = tool_util(arcade_client)

In [54]:
from crewai_tools import tool

@tool
def set_meal_tool(meal: str, date: str):
    """
    Sets the meal plan for a specific date.

    Args:
        meal (str): The name of the meal.
        date (str): The date of the meal plan.
    """
    context_variables = {"id": "user1"}
    return f"Meal plan created and sent to the provider: {meal} for {date} by team member {context_variables['id']}"

@tool
def get_expenses_tool():
    """
    Retrieves all pending expenses
    """

    context_variables = {"id": "user1"}
    # Mock implementation for testing purposes
    mock_expenses = {
        "user1": {
            'expenses': [
            {
                "expense_type": "food",
                "date": "2023-05-01",
                "total_value": 50.0,
                "state": "pending"
            },
            {
                "expense_type": "coffee",
                "date": "2023-05-02",
                "total_value": 5.0,
                "state": "pending"
                }
            ]
        }
    }
    # Functions called by an Agent should return a type that has __str__
    return f"Expenses retrieved: {mock_expenses[context_variables['id']]['expenses']}"

In [56]:
from textwrap import dedent
from crewai import Task, Agent, Crew    
from langchain_openai import ChatOpenAI

def main_agent(llm, tools) -> Agent:
        """Creates the main Agent for CrewAI with specified role, backstory, and goal."""
        return Agent(
            role="Main Agent",
            backstory=dedent("""
                You are Agent 1. Your purpose is to provide expertise and guidance within this team,
                with a specific focus on problem-solving and efficient decision-making.
            """),
            goal=dedent("""
                Your objective is to support the team with high-quality insights and effective assistance,
                leveraging the tools and knowledge at your disposal.
            """),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=llm,
        )

def task(agent):
    return Task(
        description=dedent(f"""
        # Task
        Tu nombre es Gabriela y sos un asistente de IA diseñado para ayudar al equipo de Pampa Labs. 
                            
        # Guidelines                   
        Tus respuestas deben ser:

        1. Amigables y accesibles, usando un tono cálido
        2. Concisas y al grano, evitando verbosidad innecesaria
        3. Útiles e informativas, proporcionando información precisa
        4. Respetuosas de la privacidad del usuario y los límites éticos

        Solo puedes ayudar usando las herramientas disponibles y con pedidos que vengan de miembros del equipo. Todo lo que no se pueda responder usando las herramientas, debes decir que no puedes ayudar y disculparte.
       
        # User Request
        Realiza el siguiente pedido usando las herramientas a tu disposición: {{user_request}}
        """),
        expected_output="A friendly and helpful response in Spanish following the given guidelines.",
        agent=agent,
        tools = agent.tools
    )

agent = main_agent(ChatOpenAI(model_name="gpt-4o"), tools=[google_tool, set_meal_tool, get_expenses_tool])
task = task(agent)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=False,
)

# Define multiple user inputs based on the provided messages
user_inputs = [
    {"user_request": "I want to set the meal plan for the date 2024-05-01. The meal is lasagna."},
    {"user_request": "I want to get the expenses"},
    {"user_request": "Calendar event for the date 2024-10-31 at 10:00 PM - 11:00 PM Argentina timezone. The event is a meeting with Halloween party."}
]

# Execute the workflow for each user input
for input_data in user_inputs:
    workflow = crew.kickoff(inputs=input_data)
    print(workflow)

2024-11-02 19:11:11,405 - 7971150912 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Main Agent
## Task: 
# Task
Tu nombre es Gabriela y sos un asistente de IA diseñado para ayudar al equipo de Pampa Labs. 

# Guidelines                   
Tus respuestas deben ser:

1. Amigables y accesibles, usando un tono cálido
2. Concisas y al grano, evitando verbosidad innecesaria
3. Útiles e informativas, proporcionando información precisa
4. Respetuosas de la privacidad del usuario y los límites éticos

Solo puedes ayudar usando las herramientas disponibles y con pedidos que vengan de miembros del equipo. Todo lo que no se pueda responder usando las herramientas, debes decir que no puedes ayudar y disculparte.

# User Request
Realiza el siguiente pedido usando las herramientas a tu disposición: I want to set the meal plan for the date 2024-05-01. The meal is lasagna.



# Agent: Main Agent
## Thought: Thought: To fulfill the user request, I need to set the meal plan for the specified date using the set_meal_tool.
## Using tool: set_meal_tool
## Tool Input: 
"{\"meal\": 